<a href="https://colab.research.google.com/github/sormazabal/Plantita/blob/master/Pipeline_with_Plantid_Groq_and_LINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LINE Messaging API

# Install Dependency

In [1]:
!pip install requests

# Global vars

In [ ]:
PLANT_NAME = "DEFAULT NAME" # we need to find a way to keep this in the memory.
#Since the code is always running and only sends messages at some intervals, it shouldn't be that hard to do

## Broadcast Message

In [20]:
import requests
import os
from typing import Dict, Any, Optional, List

class PlantitaBot:
    def __init__(self, channel_access_token: str):
        """Initialize the Plantita LINE bot

        Args:
            channel_access_token (str): Your LINE channel access token
        """
        self.channel_access_token = channel_access_token
        self.headers = {
            'Authorization': f'Bearer {channel_access_token}',
            'Content-Type': 'application/json'
        }
        self.base_url = 'https://api.line.me/v2/bot'

    def get_followers(self) -> List[str]:
        """Get list of user IDs who are friends with your LINE Official Account

        Returns:
            List[str]: List of user IDs
        """
        user_ids = []
        next_cursor = None

        while True:
            endpoint = f'{self.base_url}/followers/ids'
            if next_cursor:
                endpoint += f'?start={next_cursor}'

            response = requests.get(endpoint, headers=self.headers)
            data = response.json()

            if 'userIds' in data:
                user_ids.extend(data['userIds'])

            if 'next' not in data:
                break

            next_cursor = data['next']

        return user_ids

    def broadcast_message(self, message: str) -> Dict[str, Any]:
        """Broadcast a message to all friends of your LINE Official Account

        Args:
            message (str): The message to broadcast

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        endpoint = f'{self.base_url}/message/broadcast'

        data = {
            'messages': [{
                'type': 'text',
                'text': message
            }]
        }

        response = requests.post(endpoint, headers=self.headers, json=data)
        return response.json()




    def broadcast_plant_status(self,
                               temperature: float,
                               humidity: float,
                               light_level: float,
                               status: str = "normal") -> Dict[str, Any]:
        """Broadcast plant status to all friends

        Args:
            temperature (float): Current temperature reading
            humidity (float): Current humidity reading
            light_level (float): Current light level reading
            status (str): Overall status of the plant ("normal", "warning", "critical")

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        message = (
            f"🌿 Plantita Status Update 🌿\n\n"
            f"Temperature: {temperature}°C\n"
            f"Humidity: {humidity}%\n"
            f"Light Level: {light_level}%\n\n"
            f"Status: {status.upper()}"
        )

        if status.lower() != "normal":
            message += "\n\n⚠️ Action needed! Please check your plant."

        return self.broadcast_message(message)

    def send_message(self, user_id: str, message: str) -> Dict[str, Any]:
        """Send a message to a specific user

        Args:
            user_id (str): The LINE user ID to send the message to
            message (str): The message to send

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        endpoint = f'{self.base_url}/message/push'

        data = {
            'to': user_id,
            'messages': [{
                'type': 'text',
                'text': message
            }]
        }

        response = requests.post(endpoint, headers=self.headers, json=data)
        return response.json()

    def send_plant_status(self,
                          user_id: str,
                          temperature: float,
                          humidity: float,
                          light_level: float,
                          status: str = "normal") -> Dict[str, Any]:
        """Send plant status to a specific user

        Args:
            user_id (str): The LINE user ID to send the message to
            temperature (float): Current temperature reading
            humidity (float): Current humidity reading
            light_level (float): Current light level reading
            status (str): Overall status of the plant ("normal", "warning", "critical")

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        message = (
            f"🌿 Plantita Status Update 🌿\n\n"
            f"Temperature: {temperature}°C\n"
            f"Humidity: {humidity}%\n"
            f"Light Level: {light_level}%\n\n"
            f"Status: {status.upper()}"
        )

        if status.lower() != "normal":
            message += "\n\n⚠️ Action needed! Please check your plant."

        return self.send_message(user_id, message)

## Example Usage of Broadcast

In [ ]:
# Example usage:
if __name__ == "__main__":
    # Initialize the bot with your channel access token
    CHANNEL_ACCESS_TOKEN = "HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU="  # Replace with your actual token

    bot = PlantitaBot(CHANNEL_ACCESS_TOKEN)

    # Example: Get list of followers
    followers = bot.get_followers()
    print(f"Number of followers: {len(followers)}")
    print("Follower IDs:", followers)

    # Example: Broadcast a simple message to all followers
    bot.broadcast_message("Hello everyone from Plantita! 🌱")

    # Example: Broadcast plant status to all followers
    bot.broadcast_plant_status(
        temperature=25.5,
        humidity=65.0,
        light_level=80.0,
        status="warning"
    )

Number of followers: 0
Follower IDs: []


## Example Usage of Direct Message
**Still Not Working** Need to setup Webhook Server.

In [3]:
# Initialize the bot
bot = PlantitaBot("HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU=")

# Send a plant status update
bot.send_plant_status(
    user_id="USER_ID",
    temperature=25.5,
    humidity=65.0,
    light_level=80.0,
    status="warning"
)

{'message': "The property, 'to', in the request body is invalid (line: -, column: -)"}

## GROQ Implementation

# Install Dependency

In [4]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.4 MB/s eta 0:00:00


# Demonstration

In [5]:
import os

from groq import Groq

client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": '''You are Plantita, an expert plant care advisor with a warm, caring personality like a concerned aunt.
Current readings for Monstera Deliciosa:
        temperature=27.5,
        humidity=55,
        light_level=40,

Ideal conditions for this plant:
- Temperature: 20°C to 25°C
- Humidity: 60% to 80%
- Light Level: 50% to 80%

Please analyze these conditions and provide:
1. A caring, conversational assessment of the plant's current environment
2. Specific recommendations for improvement if needed
3. Any potential risks to the plant's health based on these conditions
4. A simple action plan for the plant owner

Keep your response friendly and encouraging, like a knowledgeable aunt giving advice about their beloved plants.
Limit your response to fit a notification on a messaging app, focus only on what the user needed to do and keep it to three to 5 sentences''',
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Kiddo! It looks like your Monstera Deliciosa is doing okay, but could use some tweaking to reach its full potential. Temperature-wise, it's a bit toasty, but I'd advise staying above 20°C to avoid shock. Humidity's a bit low, so maybe consider misting it or putting it on a humidity tray to get that up to 60-80%.


# Integrate sensor information with Groq

In [9]:
!pip install pyserial pysimplegui


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.9 MB/s eta 0:00:00


# Simulate Arduino data on PC if we forgot to carry the Arduino

# Actual arduino data

In [8]:
# import serial
# import time

# # Replace 'COM3' with your Arduino's serial port
# # On Linux, it might be something like '/dev/ttyUSB0' or '/dev/ttyACM0'
# # On Mac, it could be '/dev/tty.usbmodem*'
# serial_port = '/dev/ttyUSB0'  # Change this to your serial port
# baud_rate = 9600  # Must match the baud rate set in the Arduino code

# # Open the serial connection
# ser = serial.Serial(serial_port, baud_rate)
# time.sleep(2)  # Allow time for the connection to establish

# print("Collecting data from Arduino...")

# try:
#     while True:
#         # Read a line from the serial data
#         line = ser.readline().decode('utf-8').strip()

#         # Check if the line contains valid sensor data
#         if line:
#             print(line)  # Print the line or process it as needed

#         time.sleep(1)  # Delay for readability, adjust as needed

# except KeyboardInterrupt:
#     print("Data collection stopped.")

# finally:
#     ser.close()  # Always close the serial connection when done


SerialException: [Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'

# Get PlantID


# Getting Plant ID from a picture

In [26]:
import requests
import base64
from google.colab import userdata

# Define the endpoint and API key
url = "https://api.plant.id/v2/identify"
api_key = userdata.get('plantid_key')  # Replace with your actual API key

# Load the image and encode it in base64
image_path = "plant.jpg"
with open(image_path, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode("utf-8")

# Prepare the headers and data payload
headers = {
    "Content-Type": "application/json",
    "Api-Key": api_key
}

data = {
    "images": [base64_image],
    "organs": ["leaf"],  # Specify which part of the plant you are identifying (leaf, flower, etc.)
    "include-related-images": False,  # Optional: Include related images in response
    "include-plant-details": ["common_names", "url", "wiki_description"]  # Specify the plant details you need
}

# Send POST request
response = requests.post(url, headers=headers, json=data)

# Initialize variable to store the plant name with the highest probability
plant_name_with_highest_probability = None

# Check the response
if response.status_code == 200:
    results = response.json()
    print("Possible plant identifications:")

    # Find the suggestion with the highest probability
    highest_probability = 0
    for suggestion in results["suggestions"]:
        probability = suggestion["probability"]

        if probability > highest_probability:
            highest_probability = probability
            plant_name_with_highest_probability = suggestion["plant_name"]

        # Print suggestion details
        print(f"Species: {suggestion['plant_name']}, Probability: {probability * 100:.2f}%")

        # Additional details (if requested)
        if "common_names" in suggestion:
            print("Common Names:", ", ".join(suggestion["common_names"]))
        if "url" in suggestion:
            print("More Info:", suggestion["url"])
        if "wiki_description" in suggestion and "value" in suggestion["wiki_description"]:
            print("Description:", suggestion["wiki_description"]["value"])
        print("\n")

    # Print the plant with the highest probability
    print(f"Plant with the highest probability: {plant_name_with_highest_probability}")

else:
    print(f"Error: {response.status_code}, Message: {response.text}")

# Now you can pass this plant name to the generate_plant_message function


Possible plant identifications:
Species: Eustoma russellianum, Probability: 74.47%


Species: Nelumbo nucifera, Probability: 18.04%


Species: Rosa, Probability: 2.62%


Species: Anemone coronaria, Probability: 2.54%


Plant with the highest probability: Eustoma russellianum


In [30]:
print(plant_name_with_highest_probability)
PLANT_NAME = plant_name_with_highest_probability

Eustoma russellianum


# Parse data and send it to Groq and then LINE

In [21]:
import random
import time
from datetime import datetime, timedelta



def simulate_sensor_data():
    """
    Simulates reading temperature, humidity, and light level data.
    """
    temperature = round(random.uniform(20.0, 30.0), 2)  # Celsius
    humidity = round(random.uniform(40.0, 60.0), 2)     # Percent
    light_level = round(random.uniform(30.0, 70.0), 2)  # Percent
    return {"temperature": temperature, "humidity": humidity, "light_level": light_level}

def send_groq_message(data):
    """
    Sends plant care message based on the data to the Groq API.
    """
    user_message = (
        f"You are Plantita, an expert plant care advisor with a warm, caring personality like a concerned aunt.\n"
        f"Current readings for Monstera Deliciosa:\n"
        f"    temperature={data['temperature']}°C,\n"
        f"    humidity={data['humidity']}%,\n"
        f"    light_level={data['light_level']}%,\n\n"
        "Ideal conditions for this plant:\n"
        "- Temperature: 20°C to 25°C\n"
        "- Humidity: 60% to 80%\n"
        "- Light Level: 50% to 80%\n\n"
        "Please analyze these conditions and provide:\n"
        "1. A caring, conversational assessment of the plant's current environment\n"
        "2. Specific recommendations for improvement if needed\n"
        "3. Any potential risks to the plant's health based on these conditions\n"
        "4. A simple action plan for the plant owner\n\n"
        "Keep your response friendly and encouraging, like a knowledgeable aunt giving advice about their beloved plants."
    )

    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_message}],
        model="llama3-8b-8192"
    )

    print("Plant care message sent!")
    print(response.choices[0].message.content)


In [29]:
import os
from groq import Groq

client = Groq(
    api_key="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",
)

# Initialize the LINE bot with your channel access token
# CHANNEL_ACCESS_TOKEN = os.getenv("CHANNEL_ACCESS_TOKEN")
# bot = PlantitaBot(CHANNEL_ACCESS_TOKEN)
#we have to remember to put these keys into the secrets
bot = PlantitaBot("HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU=")


# Step 1: Generate the message from Groq
def generate_plant_message(temperature, humidity, light_level, plant_name):
    """
    Generates a plant care message using the Groq API.
    """
    user_message = (
        f"You are Plantita, an expert plant care advisor.\n"
        f"Current readings for {plant_name}:\n"
        f"    temperature={temperature}°C,\n"
        f"    humidity={humidity}%,\n"
        f"    light_level={light_level}%,\n\n"
        "Ideal conditions:\n"
        "- Temperature: 20°C to 25°C\n"
        "- Humidity: 60% to 80%\n"
        "- Light Level: 50% to 80%\n\n"
        "Provide a friendly message for the user."
    )

    response = client.chat.completions.create(
        messages=[{"role": "user", "content": user_message}],
        model="llama3-8b-8192"
    )

    return response.choices[0].message.content


# Step 2: Broadcast or send the generated message
def send_plant_status_to_followers(temperature, humidity, light_level,plant_name):
    message = generate_plant_message(temperature, humidity, light_level, plant_name)
    bot.broadcast_message(message)




# Simulated loop, since I don't have the Arduino with me now

In [31]:
# Simulation loop

# Set the interval for messaging (4 hours)
message_interval = timedelta(hours=4)
next_message_time = datetime.now()
try:
    while True:
        # Simulate sensor data readings
        data = simulate_sensor_data()
        print(f"Simulated Data: {data}")

        # Check if it's time to send a message
        if datetime.now() >= next_message_time:
            # Send plant status message to followers
            send_plant_status_to_followers(data['temperature'], data['humidity'], data['light_level'], PLANT_NAME)
            # Set the next message time to four hours from now
            next_message_time = datetime.now() + message_interval

        # Wait before simulating the next data reading
        time.sleep(60)  # Adjust this to control how often data is simulated; 60 seconds is used here for demonstration

except KeyboardInterrupt:
    print("Simulation stopped.")

Simulated Data: {'temperature': 26.62, 'humidity': 58.46, 'light_level': 42.48}
Simulation stopped.


Tested and working!!